In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from chemprop import data, models, nn, uncertainty
from chemprop.models import save_model, load_model
from chemprop.cli.predict import find_models
from chemprop.cli.conf import NOW
import random

# Set random seed for reproducibility
SEED = 214
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
pl.seed_everything(SEED, workers=True)

# Load Dataset
input_dir = 'D:/Postdoc-Work/Manuacripts/NLP-Organic_Solvents-Properties/Comm Chemistry/Revisions_R1/Data_ML-pred/Chemprop/VOC/'
dataset = pd.read_csv(input_dir + 'VOC-Database.csv')
print(dataset.shape)

# Remove duplicates and sort by SMILES
dataset = dataset.drop_duplicates(subset=['SMILES']).sort_values(by='SMILES')
print(dataset.shape)

mol_smiles = dataset['SMILES']
logP_values = dataset['dvap'].values.reshape(-1, 1)  # Ensure logP_values is 2D
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(mol_smiles, logP_values)]

# Split Data
mols = [d.mol for d in all_data]
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.7, 0.1, 0.2), SEED)
train_data, val_data, test_data = data.split_data_by_indices(all_data, train_indices, val_indices, test_indices)

train_dset = data.MoleculeDataset(train_data[0])
scaler = train_dset.normalize_targets()
val_dset = data.MoleculeDataset(val_data[0])
val_dset.normalize_targets(scaler)
test_dset = data.MoleculeDataset(test_data[0])

train_loader = data.build_dataloader(train_dset)
val_loader = data.build_dataloader(val_dset, shuffle=False)
test_loader = data.build_dataloader(test_dset, shuffle=False)

# Define MPNN Model
mp = nn.BondMessagePassing()
agg = nn.MeanAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.MveFFN(output_transform=output_transform)
mpnn = models.MPNN(mp, agg, ffn, batch_norm=False)

# Monitoring Setup
monitor_metric = "val_loss"
monitor_mode = "min"
print(f"Monitoring metric: {monitor_metric} with mode: {monitor_mode}")

# Checkpoint and EarlyStopping
model_output_dir = Path(f"chemprop_training/{NOW}")
checkpointing = ModelCheckpoint(
    dirpath=model_output_dir / "checkpoints",
    filename="best-{epoch}-{val_loss:.2f}",
    monitor=monitor_metric,
    mode=monitor_mode,
    save_last=True,
)

early_stop_callback = EarlyStopping(
    monitor=monitor_metric,
    patience=50,
    verbose=True,
    mode=monitor_mode
)

# Trainer
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,
    enable_progress_bar=False,
    accelerator="cpu",
    callbacks=[checkpointing, early_stop_callback],
    devices=1,
    max_epochs=2000,
    deterministic=True,
)

trainer.fit(mpnn, train_loader, val_loader)

# Report stopping epoch
print(f"Training stopped after {trainer.current_epoch + 1} epochs.")
















best_model_path = checkpointing.best_model_path
model = mpnn.__class__.load_from_checkpoint(best_model_path)
p_model = model_output_dir / "best.pt"
save_model(p_model, model)

# Load and Process Test Data
test_dset = data.MoleculeDataset(test_data[0])
test_loader = data.build_dataloader(test_dset, shuffle=False)
unc_estimator = uncertainty.MVEEstimator())

# Load trained model
model_paths = find_models([model_output_dir])
models = [load_model(model_path, multicomponent=False) for model_path in model_paths]
trainer = pl.Trainer(logger=False, enable_progress_bar=True, accelerator="cpu", devices=1)

test_predss, test_uncss = unc_estimator(test_loader, models, trainer)
test_preds = test_predss.mean(0)

df_test = pd.DataFrame(
    {
        "smiles": test_dset.smiles,
        "target": test_dset.Y.reshape(-1),
        "pred": test_preds.reshape(-1),
    }
)

df_test.to_csv(f'MPFF_model-Enthalpy-Vap_{SEED}seed-SMILES_CommChem_70-10-20_Early.csv', index=False)


NameError: name 'random' is not defined

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor

# Load dataset
input_dir = 'D:/Postdoc-Work/Results/ML_Codes/Scikit-Learn_ML/Models_MM/Partition-Coeff_ML/Input-Data/'
dataset = pd.read_csv(input_dir + 'logP_dataset_Kaggle+NLP_Canonical_SMILES_Atom-Count.csv')

# Remove duplicate SMILES
dataset = dataset.drop_duplicates(subset=['Canonical_SMILES']).sort_values(by='Canonical_SMILES')

mol_smiles = dataset['Canonical_SMILES']
logP = dataset['Exp_logp']

# Function to compute RDKit descriptors
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors = []
    for mol in mols:
        mol = Chem.AddHs(mol)  # Add hydrogens
        descriptors = calc.CalcDescriptors(mol)  # Calculate descriptors
        Mol_descriptors.append(descriptors)
    
    return Mol_descriptors, desc_names 

# Compute RDKit features
Mol_descriptors, desc_names = RDkit_descriptors(mol_smiles)
Mol_descriptors = pd.DataFrame(Mol_descriptors, columns=desc_names)

# Merge with dataset
dataset = pd.concat([dataset, Mol_descriptors], axis=1)

# Remove zero-value and NaN columns
dataset = dataset.loc[:, (dataset != 0).any(axis=0)]
dataset = dataset.dropna(axis=1)

# Remove highly correlated features
def correlation(Mol_descriptors, threshold):
    col_corr = set()
    corr_matrix = Mol_descriptors.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

corr_features = correlation(Mol_descriptors, 0.8)
Mol_descriptors = Mol_descriptors.drop(corr_features, axis=1)

# K-Fold Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=41)

cnt = 1
for train_index, test_index in kf.split(Mol_descriptors, logP):
    print(f'Fold {cnt}: Train {len(train_index)}, Test {len(test_index)}')

    # Train-test split
    X_train, X_test = Mol_descriptors.iloc[train_index], Mol_descriptors.iloc[test_index]
    y_train, y_test = logP.iloc[train_index], logP.iloc[test_index]
    
    # Train three CatBoost models for quantile regression
    model_q05 = CatBoostRegressor(loss_function="Quantile:alpha=0.05", verbose=200).fit(X_train, y_train)
    model_q50 = CatBoostRegressor(loss_function="Quantile:alpha=0.50", verbose=200).fit(X_train, y_train)
    model_q95 = CatBoostRegressor(loss_function="Quantile:alpha=0.95", verbose=200).fit(X_train, y_train)


    # Predictions
    y_pred_q05 = model_q05.predict(X_test)
    y_pred_q50 = model_q50.predict(X_test)  # Median prediction
    y_pred_q95 = model_q95.predict(X_test)

    # Compute uncertainty (prediction interval width)
    uncertainty = y_pred_q95 - y_pred_q05

    # Evaluate model performance
    mae = mean_absolute_error(y_test, y_pred_q50)
    rmse = math.sqrt(mean_squared_error(y_test, y_pred_q50))
    r2 = r2_score(y_test, y_pred_q50)
    
    print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.3f}")

    # Save results
    results_df = pd.DataFrame({
        'SMILES': mol_smiles.iloc[test_index],
        'Exp_logP': y_test,
        'Pred_logP': y_pred_q50,
        'Lower_Bound_5%': y_pred_q05,
        'Upper_Bound_95%': y_pred_q95,
        'Uncertainty': uncertainty
    })
    results_df.to_csv(f'logP_CatBoost_Quantile_Uncertainty_Fold{cnt}.csv', index=False)

    # Visualization
    plt.figure(figsize=(6, 5))
    plt.plot(y_test, y_test, color='gray', linestyle='--')
    plt.scatter(y_test, y_pred_q50, c=uncertainty, cmap='coolwarm', edgecolor='k', marker="D", s=80)
    plt.colorbar(label='Uncertainty')
    
    plt.xlabel('Experimental logP', fontsize=14)
    plt.ylabel('Predicted logP', fontsize=14)
    plt.title(f'Fold {cnt}: logP Predictions with Uncertainty', fontsize=14)
    plt.grid(True)
    plt.show()

    cnt += 1
